<b style="color:red">**NOTE:**</b> Каждый шаг делал с учетом предидущего т.е. на каждом шаге признаки только добавлялись - возможно я неверно понял и нужно было каждый шаг оценивать отдельно по отношению к базовой модели ...

# ДЗ №3

Если будете решать ДЗ на отдельном датасете (не датасет из соревнования) - то использовать датасет из ДЗ2, IEEE-Fraud-Detection Challenge.

## Библиотеки и загрузка данных

In [1]:
import warnings

from datetime import datetime, timedelta

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import catboost as cb

warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
def fit_catboost(x_train, y_train, x_valid, y_valid, params, verbose=100):
    eval_set = [(x_valid, y_valid)]
    model = cb.CatBoostClassifier(**params)
    model.fit(X=x_train, y=y_train,
              eval_set=eval_set,
              cat_features=categorical,
              verbose=verbose)

    return model

def evaluate_model(model, *args):
    eval_data = [(args[i], args[i+1]) for i in range(0, len(args), 2)]
    if eval_data:
        for sample, target in eval_data:
            y_pred = model.predict_proba(sample)[:, 1]
            score = roc_auc_score(target, y_pred)
            print(f"score = {score:0.6f}")

def prepare_data(X, categorical, to_drop):
    X_transformed = X.copy()
    to_drop = set(X.columns) & set(to_drop)
    if to_drop:
        X_transformed = X_transformed.drop(to_drop, axis=1)
    X_transformed[categorical] = X_transformed[categorical].astype(str)
    return X_transformed

In [3]:
train = pd.read_csv('../data/assignment_train.csv.zip', compression='zip')
test = pd.read_csv('../data/assignment_test.csv.zip', compression='zip')

In [4]:
to_drop = [
    "TransactionID",
    "TransactionDT",
    "isFraud",
]

categorical = train.select_dtypes(include=["object"]).columns.tolist()

In [5]:
XT, XV, y_train, y_valid = train_test_split(
    train, train["isFraud"], 
    train_size=0.7, random_state=42, 
    shuffle=True
)

## Задание 0: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. 

Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

In [6]:
cb_sm = {
    "n_estimators": 1000,
    "learning_rate": 0.01,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 30,
    "thread_count": 6,
    "random_seed": 42
}

def test_model(x_train, x_valid, x_test):
    print(f'Columns = {x_train.shape[1]}\n')
    model = fit_catboost(x_train, y_train, x_valid, y_valid, cb_sm, 100)
    evaluate_model(model, x_train, y_train, x_valid, y_valid, x_test, test["isFraud"])

In [7]:
x_train = prepare_data(XT, categorical=categorical, to_drop=to_drop)
x_valid = prepare_data(XV, categorical=categorical, to_drop=to_drop)
x_test = prepare_data(test, categorical=categorical, to_drop=to_drop)

test_model(x_train, x_valid, x_test)

Columns = 391

0:	test: 0.6192122	best: 0.6192122 (0)	total: 239ms	remaining: 3m 59s
100:	test: 0.8117806	best: 0.8117806 (100)	total: 19.9s	remaining: 2m 57s
200:	test: 0.8336717	best: 0.8336717 (200)	total: 36.5s	remaining: 2m 24s
300:	test: 0.8373416	best: 0.8374015 (298)	total: 52.5s	remaining: 2m 1s
400:	test: 0.8403872	best: 0.8403872 (400)	total: 1m 8s	remaining: 1m 41s
500:	test: 0.8440294	best: 0.8440294 (500)	total: 1m 25s	remaining: 1m 24s
600:	test: 0.8463114	best: 0.8463114 (600)	total: 1m 41s	remaining: 1m 7s
700:	test: 0.8482284	best: 0.8482284 (700)	total: 1m 57s	remaining: 50.1s
800:	test: 0.8502504	best: 0.8502504 (800)	total: 2m 13s	remaining: 33.2s
900:	test: 0.8515970	best: 0.8516079 (896)	total: 2m 29s	remaining: 16.5s
999:	test: 0.8527042	best: 0.8527224 (993)	total: 2m 45s	remaining: 0us

bestTest = 0.85272243
bestIteration = 993

Shrink model to first 994 iterations.
score = 0.893857
score = 0.852722
score = 0.861933


## Задание 1: признак TransactionDT - это смещение в секундах относительно базовой даты. 

Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [8]:
def step1(X):
    base = datetime.fromisoformat('2017-12-01')
    res = X[['TransactionDT']].copy()
    p = res['TransactionDT'].apply(lambda x: base + timedelta(seconds=x))
    res['year'] = p.dt.year
    res['month'] = p.dt.month
    res['day'] = p.dt.day
    res['day_of_week'] = p.dt.weekday
    res['hour'] = p.dt.hour
    return res.drop('TransactionDT', axis=1)

x_train = pd.concat([x_train, step1(XT)], axis=1)
x_valid = pd.concat([x_valid, step1(XV)], axis=1)
x_test = pd.concat([x_test, step1(test)], axis=1)

test_model(x_train, x_valid, x_test)

Columns = 396

0:	test: 0.6174489	best: 0.6174489 (0)	total: 166ms	remaining: 2m 46s
100:	test: 0.8121317	best: 0.8122543 (91)	total: 19.5s	remaining: 2m 53s
200:	test: 0.8333778	best: 0.8335564 (188)	total: 35.9s	remaining: 2m 22s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8351618372
bestIteration = 235

Shrink model to first 236 iterations.
score = 0.871177
score = 0.835162
score = 0.849027


> **Вывод:** насколько я понял особого прироста качества эти данные не дали, возможно даже ухудшили

## Задание 2: сгруппировать данные по card1 и посчитать среднюю сумму транзакции.

Добавить в качестве признака в набор данных. Посчитать разницу между суммой транзакцией пользователя и средней суммой транзакции по данному типу card1. Построить отношение этих признаков. Повторить процедуру для всех card.

In [9]:
def step2(X):
    cards = [f'card{i}' for i in range(1,7)]
    res = X[['TransactionAmt']+cards].copy()
    for i in cards:
        m = res.groupby(i)['TransactionAmt'].mean()
        res[f'{i}_mean'] = res[i].map(m)
        res[f'{i}_diff'] = res['TransactionAmt'] - res[f'{i}_mean']
        res[f'{i}_rel'] = res['TransactionAmt'] / res[f'{i}_mean']
    return res.drop(['TransactionAmt']+cards, axis=1)

x_train = pd.concat([x_train, step2(XT)], axis=1)
x_valid = pd.concat([x_valid, step2(XV)], axis=1)
x_test = pd.concat([x_test, step2(test)], axis=1)

test_model(x_train, x_valid, x_test)

Columns = 414

0:	test: 0.6111935	best: 0.6111935 (0)	total: 169ms	remaining: 2m 48s
100:	test: 0.8237884	best: 0.8237884 (100)	total: 16.5s	remaining: 2m 26s
200:	test: 0.8372628	best: 0.8377381 (191)	total: 32.8s	remaining: 2m 10s
300:	test: 0.8406375	best: 0.8407166 (298)	total: 49.4s	remaining: 1m 54s
400:	test: 0.8438370	best: 0.8439751 (399)	total: 1m 5s	remaining: 1m 38s
500:	test: 0.8471846	best: 0.8471846 (500)	total: 1m 22s	remaining: 1m 22s
600:	test: 0.8497551	best: 0.8497551 (600)	total: 1m 48s	remaining: 1m 11s
700:	test: 0.8523476	best: 0.8523476 (700)	total: 2m 10s	remaining: 55.6s
800:	test: 0.8545498	best: 0.8545498 (800)	total: 2m 27s	remaining: 36.7s
900:	test: 0.8560212	best: 0.8560212 (900)	total: 2m 46s	remaining: 18.3s
999:	test: 0.8574031	best: 0.8574031 (999)	total: 3m 4s	remaining: 0us

bestTest = 0.8574031111
bestIteration = 999

score = 0.897983
score = 0.857403
score = 0.865802


> **Вывод:** добавление таких признаков улучшило качество модели

## Задание 3: преобразовать признаки card_1 - card_6 с помощью Frequency Encoding;

In [10]:
def step3(X):
    cards = [f'card{i}' for i in range(1,7)]
    res = X[cards].copy()
    for i in cards:
        m = res[i].value_counts(normalize=True)
        res[f'{i}_freq'] = res[i].map(m)
    return res.drop(cards, axis=1)

x_train = pd.concat([x_train, step3(XT)], axis=1)
x_valid = pd.concat([x_valid, step3(XV)], axis=1)
x_test = pd.concat([x_test, step3(test)], axis=1)

test_model(x_train, x_valid, x_test)

Columns = 420

0:	test: 0.6433701	best: 0.6433701 (0)	total: 189ms	remaining: 3m 8s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8153935385
bestIteration = 47

Shrink model to first 48 iterations.
score = 0.836277
score = 0.815394
score = 0.827628


> **Вывод:** судя по всему особого прироста качества эти данные не дали, даже возможно ухудшили

## Задание 4: преобразовать признак TransactionAmt в логариф признака, выделить дробную часть и целую часть в отдельные признаки.

In [11]:
def step4(X):
    res = X[['TransactionAmt']].copy()
    m = np.log(X['TransactionAmt'])
    res['TransactionAmt_ceil'] = m // 1
    res['TransactionAmt_floor'] = m - res['TransactionAmt_ceil']
    return res.drop('TransactionAmt', axis=1)

x_train = pd.concat([x_train, step4(XT)], axis=1)
x_valid = pd.concat([x_valid, step4(XV)], axis=1)
x_test = pd.concat([x_test, step4(test)], axis=1)

test_model(x_train, x_valid, x_test)

Columns = 422

0:	test: 0.6366957	best: 0.6366957 (0)	total: 180ms	remaining: 2m 59s
100:	test: 0.8189844	best: 0.8189844 (100)	total: 22.2s	remaining: 3m 17s
200:	test: 0.8381693	best: 0.8381693 (200)	total: 44.6s	remaining: 2m 57s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.8411261294
bestIteration = 235

Shrink model to first 236 iterations.
score = 0.873915
score = 0.841126
score = 0.849746


> **Вывод:** если смотреть на предидущий шаг и на этот то модель явно улучшилась

## Задание 5: для числовых признаков построить PCA-признаки, добавить их к основной части датасета.

In [12]:
num = train.select_dtypes(exclude=["object"]).columns.tolist()
num = list(filter(lambda x: x not in to_drop, num))

sc = StandardScaler()
pca = PCA(n_components=2)
pca.fit(np.nan_to_num(sc.fit_transform(XT[num])))

PCA(n_components=2)

In [13]:
def step5(X):
    res = X[num].copy()
    p = pca.transform(np.nan_to_num(sc.transform(res)))
    res['pca_1'] = p[:, 0]
    res['pca_2'] = p[:, 1]
    return res.drop(num, axis=1)

x_train = pd.concat([x_train, step5(XT)], axis=1)
x_valid = pd.concat([x_valid, step5(XV)], axis=1)
x_test = pd.concat([x_test, step5(test)], axis=1)

test_model(x_train, x_valid, x_test)

Columns = 424

0:	test: 0.6582504	best: 0.6582504 (0)	total: 186ms	remaining: 3m 5s
100:	test: 0.8231825	best: 0.8231825 (100)	total: 19.5s	remaining: 2m 53s
200:	test: 0.8389116	best: 0.8389116 (200)	total: 39.6s	remaining: 2m 37s
300:	test: 0.8427131	best: 0.8427300 (295)	total: 1m	remaining: 2m 19s
400:	test: 0.8462938	best: 0.8462938 (400)	total: 1m 21s	remaining: 2m 1s
500:	test: 0.8493975	best: 0.8493975 (500)	total: 1m 41s	remaining: 1m 40s
600:	test: 0.8522042	best: 0.8522042 (600)	total: 2m	remaining: 1m 20s
700:	test: 0.8553175	best: 0.8553175 (700)	total: 2m 17s	remaining: 58.8s
800:	test: 0.8570205	best: 0.8570205 (800)	total: 2m 34s	remaining: 38.3s
900:	test: 0.8581881	best: 0.8581881 (900)	total: 2m 57s	remaining: 19.5s
999:	test: 0.8590602	best: 0.8590602 (999)	total: 3m 16s	remaining: 0us

bestTest = 0.8590601825
bestIteration = 999

score = 0.896582
score = 0.859060
score = 0.864252


> **Вывод:** данные признаки явно улучшают модель, думаю обобщающие PCA признаки перекрывают остальные признаки 

## Задание 6: использовать критерий отбора признаков на основе перестановок для отбора признаков, которые положительно влияют на перформанс модели. 
Переобучить модель и сделать выводы о полученном качестве алгоритма.

## Задание 7: (опциональное) испольовать SHAP для оценки важности признаков. 

Использовать SHAP для отбора признаков, проанализировать поведение модели на одном (любом) примере, посмотреть какой вклад каждый признак вносит в итоговый прогно. Проанализировать резульаты.